In [1]:
from selenium import webdriver
from selenium.common.exceptions import WebDriverException
from selenium.common.exceptions import SessionNotCreatedException

from time import sleep
from selenium.webdriver.common.keys import Keys
from parsel import Selector

from bs4 import BeautifulSoup  
from urllib.request import urlopen as uReq

import pandas as pd
from datetime import datetime
from datetime import date


In [2]:
# specifies driver
path = 'F:\Google drive - theanantagr\Colab Notebooks\Expedia'
chromepath = path.replace('\\',"/")+'/chromedriver.exe'

driver = webdriver.Chrome(chromepath)
sleep(0.5)

In [3]:
driver.get('https://www.expedia.com/Hotel-Search?adults=2&d1=2021-02-17&d2=2021-02-18&destination=Greensboro%20%28and%20vicinity%29%2C%20North%20Carolina%2C%20United%20States%20of%20America&endDate=2021-02-18&latLong=36.072517%2C-79.791512&regionId=602968&rooms=1&semdtl=&sort=RECOMMENDED&startDate=2021-02-17&theme=&useRewards=false&userIntent')
sleep(4)

In [4]:
lenOfPage = driver.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
match=False
while(match==False):
    lastCount = lenOfPage
    sleep(1)
    lenOfPage = driver.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
        
    if lastCount==lenOfPage:
        match=True
while True:
    try:
        driver.find_element_by_xpath('/html[1]/body[1]/div[2]/div[1]/div[1]/div[1]/div[1]/div[1]/div[1]/main[1]/div[2]/div[1]/div[2]/section[3]/button[1]').click()
        # head_driver.find_element_by_xpath('/html/body/main/div/section/button').click()

    except:
        break
sleep(1)

In [5]:
pageSource = driver.page_source
lxml_soup = BeautifulSoup(pageSource, 'lxml')

# searching for all hotels
hotel_container = lxml_soup.findAll('li', {'class': 'uitk-spacing listing uitk-spacing-margin-blockstart-three horizontal'})
sleep(2)


In [24]:
hotel_names = []
hotel_location = []
hotel_links = []
hotel_tarrif = []
hotel_total_charge = []
hotel_rating = []
hotel_status = []
num_reviews = []


In [29]:
for hotel in hotel_container:
    
    #Scrapping info from main search page
    name = hotel.find('h3', {'class': 'truncate-lines-2 all-b-padding-half pwa-theme--grey-900 uitk-type-heading-500'}).text
    hotel_names.append(name)
    
    location = hotel.find('div', {'data-test-id': 'content-hotel-neighborhood'}).text
    hotel_location.append(location)

    link = hotel.find('a', {'class': 'listing__link uitk-card-link'})['href']
    hotel_links.append('https://www.expedia.com' + link)

    rating = hotel.find('span', {'data-stid': 'content-hotel-reviews-rating'}).text
    hotel_rating.append(rating)

    # status = hotel.find('span', {'data-stid': 'content-hotel-reviews-superlative'}).text
    # hotel_status.append(status)

    num_review = hotel.find('span', {'data-stid': 'content-hotel-reviews-total'}).text
    num_reviews.append(num_review)

    tarrif = hotel.find('span', {'data-stid': 'content-hotel-lead-price'}).text
    hotel_tarrif.append(tarrif)

In [30]:
df = pd.DataFrame()

In [31]:
df['Hotel Name'] = hotel_names
df['Location'] = hotel_location
df['Tarrif'] = hotel_tarrif
df['Ratings'] = hotel_rating
df['Number of reviews'] = num_reviews
df['Website'] = hotel_links


In [32]:
df.head()

,Hotel Name,Location,Tarrif,Ratings,Number of reviews,Website
0,Wingate by Wyndham High Point,High Point,$93,4.1/5,(996 reviews),https://www.expedia.com/Greensboro-Hotels-Wing...
1,Hawthorn Suites by Wyndham Greensboro,Greensboro,$71,4.0/5,(888 reviews),https://www.expedia.com/Greensboro-Hotels-Hawt...
2,Proximity Hotel,Greensboro,$256,4.7/5,(347 reviews),https://www.expedia.com/Greensboro-Hotels-Prox...
3,O.Henry Hotel,Greensboro,$269,4.8/5,(289 reviews),https://www.expedia.com/Greensboro-Hotels-OHen...
4,Wingate by Wyndham - Greensboro,Greensboro,$78,4.5/5,"(1,002 reviews)",https://www.expedia.com/Greensboro-Hotels-Wing...


In [33]:
df.to_csv (r'expedia_info.csv', index = False, header=True)
